In [175]:
import pandas as pd
import datetime
from datetime import timedelta
from flask_sqlalchemy import *#SQLAlchemy
from sqlalchemy import create_engine
import numpy as np

In [176]:
df = pd.read_csv("https://docs.google.com/spreadsheets/d/1k921SCuARlqhDQsMh4F_R0-Eyc5tTCALJKf5k1HDRkc/export?format=csv&gid=201423930")

In [177]:
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
df["Timestamp"] = df.apply(lambda x: (x["Timestamp"]+datetime.timedelta(hours=4)).date(),axis=1)
df

,Timestamp,이름,암호,점수
0,2019-12-04,윤동규,1234,200
1,2019-12-04,윤동규,1234,178
2,2019-12-04,윤동규,1515,188
3,2019-12-04,평복재,1515,199
4,2019-12-04,평복재,1515,300
5,2019-12-04,조재현,123,201
6,2019-12-04,곽동준,456,148
7,2019-12-04,평복123,21,123
8,2019-12-04,윤동규,1234,151
9,2019-12-04,곰돌이,258,150


In [178]:
members = pd.read_csv("https://docs.google.com/spreadsheets/d/1A2-4620fVl4PcclTUuhzaroaRo9iKB2iz1lTGSwWBQI/export?format=csv&gid=0")

In [181]:
df["c"]= df["이름"].astype(str) + df["암호"].astype(str)
members["c"] = members["name"].astype(str) + members["pw"].astype(str)

In [184]:
df = df[df["c"].isin(members["c"])].drop(columns = ["c"])
df

,Timestamp,이름,암호,점수
0,2019-12-04,윤동규,1234,200
1,2019-12-04,윤동규,1234,178
3,2019-12-04,평복재,1515,199
4,2019-12-04,평복재,1515,300
5,2019-12-04,조재현,123,201
6,2019-12-04,곽동준,456,148
8,2019-12-04,윤동규,1234,151
11,2019-12-05,허만,169,151


In [185]:
df.drop(columns=["암호"],inplace = True)

In [186]:
df["점수"] = "," + df["점수"].astype(str)

In [187]:
df = df.pivot_table(index="이름",values="점수",columns="Timestamp",aggfunc="sum")
df.sort_index(axis=1,ascending=False,inplace = True)

In [188]:
df["scores"] = df.fillna("").sum(axis=1).str[1:]

In [189]:
df["scores"] = df["scores"].str.split(",")

In [190]:
df["scores"] = df.apply(lambda x: [int(i) for i in x["scores"]],axis=1)
df["games"] = df.apply(lambda x: len(x["scores"]),axis=1)
df["avg"] = (df.apply(lambda x: sum(x["scores"]),axis=1)/df["games"]).astype(int)
df["stdev"] = df.apply(lambda x: np.std(x["scores"]),axis=1).round(1)
df.drop(columns = ["scores"],inplace = True)
df.reset_index(inplace=True)
df.sort_values("이름",ascending = True, inplace = True)
df.sort_values("games",ascending = False, inplace = True)
df.set_index(["이름","games","avg","stdev"],inplace = True)

In [191]:
df

,,,Timestamp,2019-12-05,2019-12-04
이름,games,avg,stdev,,
윤동규,3,176,20.0,NaN,",200,178,151"
평복재,2,249,50.5,NaN,",199,300"
곽동준,1,148,0.0,NaN,",148"
조재현,1,201,0.0,NaN,",201"
허만,1,151,0.0,",151",NaN
